# Setup

In [1]:
!pip -q install openai
!pip -q install langchain
!pip -q install langchain_openai # >= 0.1.0
!pip -q install langchain_experimental
!pip -q install sentence-transformers
!pip -q install tiktoken
!pip -q install pypdf

!pip -q install docarray
!pip -q install chromadb
!pip -q install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [8]:
!python -c "import openai; print('openai:', openai.__version__)"
!python -c "import langchain; print('langchain:', langchain.__version__)"

openai: 1.10.0
langchain: 0.1.4


In [9]:
#API KEY 저장을 위한 os 라이브러리 호출
import os
from google.colab import userdata

#OPENAI API키 저장
#API KEY 발급 페이지: https://platform.openai.com/docs/guides/gpt/completions-api
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 모듈 임포트 및 임베딩 모델 생성

In [11]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

In [12]:
# 각 문서 조각을 임베딩할 임베딩 모델 선언
# embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

## DocArray

- DocArrayInMemorySearch는 메모리에 문서를 저장

- 데이터베이스 서버를 실행하고 싶지 않은 소규모 데이터 세트에 적합한 시작점

- https://python.langchain.com/docs/integrations/vectorstores/docarray_in_memory

- https://docs.docarray.org/

### 텍스트 로딩

In [ ]:
# pdf 파일을 페이별로 잘라서 로딩
loader= PyPDFLoader("문무대왕함.pdf")
pdf_pages = loader.load_and_split()

# 페이지별로 읽혀진 pdf를 다시 문서조각으로 자름
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0)
docs_inmem = text_splitter.split_documents(pdf_pages)

### 벡터 db 생성 및 유사도 검색

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [ ]:
inmem_db = DocArrayInMemorySearch.from_documents(docs_inmem, embeddings)
inmem_db

In [ ]:
query = "문무대왕함의 특징은?"

# score = 1 - cos(theta) 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = inmem_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs

[(Document(page_content="오늘 '문무대왕함 '의 탄생으로  우리 해군은  활동영역이  더욱 넓어지게  되었습니다 . 遠海에서의  작\n전능력이  크게 향상되고 , 입체적인  대함, 대잠, 대공 작전능력도  한층 강화됩니다 . 최첨단  무기체\n계와 스텔스  기능, 그리고  최신의  자동화된  지휘통제  시스템도  '문무대왕함 '의 자랑입니다 .", metadata={'source': '문무대왕함.pdf', 'page': 0}),
  0.66217961095113),
 (Document(page_content="대한민국  해군의  무궁한  발전과  '문무대왕함 '의 武運長久를  기원합니다 .", metadata={'source': '문무대왕함.pdf', 'page': 1}),
  0.644339711434625),
 (Document(page_content='친애하는  해군장병  여러분 , \n \n그리고  현대중공업  임직원과  내외 귀빈 여러분 , \n \n오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최\n신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의', metadata={'source': '문무대왕함.pdf', 'page': 0}),
  0.6342732301885562),
 (Document(page_content="동아시아의  바다를  제패했던  장보고  대사의  자랑스런  역사가  있습니다 . 이제는  우리가  그 역사를  \n새롭게  이어가야  합니다 . 5대양으로  항진해  나아가는  '문무대왕함 '에는 대양해군의  웅대한  비전이  \n담겨 있습니다 . 삼국통일의  위업을  이룩하고  죽어서도  나라를  지켜내겠다던  문무대왕의  기개와", metadata={'source': '문무대왕함.pdf', 'page': 1}),
  0.5747157734798365)]

## Chroma

- AI 네이티브 오픈소스 벡터 데이터베이스

- DB로써 완전한 기능을 제공 비슷한 유료 툴로 pinecone, weaviate

- https://python.langchain.com/docs/integrations/vectorstores/chroma

- https://www.trychroma.com/

In [ ]:
# db로써의 기능을 조금 더 알아서 추가할 필요 있음
# 2024.01.23

### 텍스트 로딩

In [ ]:
# pdf 파일을 페이별로 잘라서 로딩
loader= PyPDFLoader("문무대왕함.pdf")
pdf_pages = loader.load_and_split()

# 페이지별로 읽혀진 pdf를 다시 문서조각으로 자름
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0)
docs_for_chroma = text_splitter.split_documents(pdf_pages)

### 벡터 db 생성 및 유사도 검색

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
# 다시 만들면 기존에 있던 레코드에 추가되기 때문에 비워내고 다시 만듬
Chroma().delete_collection()
chroma_db = Chroma.from_documents(docs_for_chroma, embeddings)
chroma_db

In [ ]:
query = "문무대왕함의 특징은?"

# # L2 distance 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = chroma_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs, len(search_docs)

([(Document(page_content="오늘 '문무대왕함 '의 탄생으로  우리 해군은  활동영역이  더욱 넓어지게  되었습니다 . 遠海에서의  작\n전능력이  크게 향상되고 , 입체적인  대함, 대잠, 대공 작전능력도  한층 강화됩니다 . 최첨단  무기체\n계와 스텔스  기능, 그리고  최신의  자동화된  지휘통제  시스템도  '문무대왕함 '의 자랑입니다 .", metadata={'page': 0, 'source': '문무대왕함.pdf'}),
   279.55120849609375),
  (Document(page_content='친애하는  해군장병  여러분 , \n \n그리고  현대중공업  임직원과  내외 귀빈 여러분 , \n \n오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최\n신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의', metadata={'page': 0, 'source': '문무대왕함.pdf'}),
   297.8609313964844),
  (Document(page_content="대한민국  해군의  무궁한  발전과  '문무대왕함 '의 武運長久를  기원합니다 .", metadata={'page': 1, 'source': '문무대왕함.pdf'}),
   303.2062683105469),
  (Document(page_content="동아시아의  바다를  제패했던  장보고  대사의  자랑스런  역사가  있습니다 . 이제는  우리가  그 역사를  \n새롭게  이어가야  합니다 . 5대양으로  항진해  나아가는  '문무대왕함 '에는 대양해군의  웅대한  비전이  \n담겨 있습니다 . 삼국통일의  위업을  이룩하고  죽어서도  나라를  지켜내겠다던  문무대왕의  기개와", metadata={'page': 1, 'source': '문무대왕함.pdf'}),
   343.9403076171875)],
 4)

In [ ]:
# .get()으로 모든 내용 가져오기
chroma_db.get()

{'ids': ['f6883bec-b9f4-11ee-93f3-0242ac1c000c',
  'f6883d22-b9f4-11ee-93f3-0242ac1c000c',
  'f6883dae-b9f4-11ee-93f3-0242ac1c000c',
  'f6883e30-b9f4-11ee-93f3-0242ac1c000c',
  'f6883ea8-b9f4-11ee-93f3-0242ac1c000c',
  'f6883f20-b9f4-11ee-93f3-0242ac1c000c',
  'f6883f8e-b9f4-11ee-93f3-0242ac1c000c',
  'f6884006-b9f4-11ee-93f3-0242ac1c000c',
  'f688407e-b9f4-11ee-93f3-0242ac1c000c',
  'f688415a-b9f4-11ee-93f3-0242ac1c000c',
  'f68841d2-b9f4-11ee-93f3-0242ac1c000c',
  'f6884240-b9f4-11ee-93f3-0242ac1c000c',
  'f68842b8-b9f4-11ee-93f3-0242ac1c000c',
  'f6884330-b9f4-11ee-93f3-0242ac1c000c',
  'f68843bc-b9f4-11ee-93f3-0242ac1c000c'],
 'embeddings': None,
 'metadatas': [{'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 1, 'source': '문무대왕함.pdf'},
  {'page': 1, 'source': 

In [ ]:
print(chroma_db.get()['documents'][0])
print('-'*100)
print(chroma_db.get()['documents'][1])

친애하는  해군장병  여러분 , 
 
그리고  현대중공업  임직원과  내외 귀빈 여러분 , 
 
오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최
신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의
----------------------------------------------------------------------------------------------------
국방과학기술과  자주국방의  의지를  상징하고  있습니다 . 날로 성장하는  '필승 해군'과 우리 조선산
업의 뛰어난  역량에  대해서  무한한  자부심을  느낍니다 . 정말 자랑스럽고  마음 든든합니다 . 
 
그동안  밤낮없이  애써주신  현대중공업의  기술진과  근로자  여러분 , 그리고  해군과  국방과학연구소


In [ ]:
# chromadb 내부 컬렉션
# 이게 쌩 chromadb 객체임
type(chroma_db._collection)

chromadb.api.models.Collection.Collection

In [ ]:
doc = "배수량 : 4400t(만재 5520 t)  크기 : 150m(길이)×17.4m(선폭)×7.3m(흘수선)  속력 : 29노트(54km/h)  항속거리 : 10,200km (5,500해리)  승조원 : 300명"
chroma_db._collection.add(
    documents=[doc],
    embeddings = [embeddings.embed_documents(doc)[0]],
    metadatas=[{'page': -1, 'source': '나무위키'},],
    ids=['aux-002']
)

In [ ]:
# len(chroma_db.get(include=['embeddings'])['embeddings'])
chroma_db.get()

{'ids': ['aux-001',
  'aux-002',
  'f6883bec-b9f4-11ee-93f3-0242ac1c000c',
  'f6883d22-b9f4-11ee-93f3-0242ac1c000c',
  'f6883dae-b9f4-11ee-93f3-0242ac1c000c',
  'f6883e30-b9f4-11ee-93f3-0242ac1c000c',
  'f6883ea8-b9f4-11ee-93f3-0242ac1c000c',
  'f6883f20-b9f4-11ee-93f3-0242ac1c000c',
  'f6883f8e-b9f4-11ee-93f3-0242ac1c000c',
  'f6884006-b9f4-11ee-93f3-0242ac1c000c',
  'f688407e-b9f4-11ee-93f3-0242ac1c000c',
  'f688415a-b9f4-11ee-93f3-0242ac1c000c',
  'f68841d2-b9f4-11ee-93f3-0242ac1c000c',
  'f6884240-b9f4-11ee-93f3-0242ac1c000c',
  'f68842b8-b9f4-11ee-93f3-0242ac1c000c',
  'f6884330-b9f4-11ee-93f3-0242ac1c000c',
  'f68843bc-b9f4-11ee-93f3-0242ac1c000c'],
 'embeddings': None,
 'metadatas': [{'page': -1, 'source': '나무위키'},
  {'page': -1, 'source': '나무위키'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'page': 0, 'source': '문무대왕함.pdf'},
  {'pa

In [ ]:
query = "성능"

# L2 distance 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = chroma_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs, len(search_docs)

([(Document(page_content='감사합니다 .', metadata={'page': 2, 'source': '문무대왕함.pdf'}),
   522.1806030273438),
  (Document(page_content="리고 이번에  고도의  기술력이  필요한  4천톤급의  구축함을  만들어  낸 것입니다 . 머지 않아 '이지스\n'(Aegis) 체계까지  갖춘 7천톤급의  구축함  시대도  실현해  낼 것입니다 . \n \n앞으로도  조선산업의  역할은  계속될  것입니다 . 반도체와  정보통신 , 자동차와  함께 한국을  대표하", metadata={'page': 1, 'source': '문무대왕함.pdf'}),
   538.9776611328125),
  (Document(page_content='배수량 : 4400t(만재 5520 t)  크기 : 150m(길이)×17.4m(선폭)×7.3m(흘수선)  속력 : 29노트(54km/h)  항속거리 : 10,200km (5,500해리)  승조원 : 300명', metadata={'page': -1, 'source': '나무위키'}),
   552.7227783203125),
  (Document(page_content="국방과학기술과  자주국방의  의지를  상징하고  있습니다 . 날로 성장하는  '필승 해군'과 우리 조선산\n업의 뛰어난  역량에  대해서  무한한  자부심을  느낍니다 . 정말 자랑스럽고  마음 든든합니다 . \n \n그동안  밤낮없이  애써주신  현대중공업의  기술진과  근로자  여러분 , 그리고  해군과  국방과학연구소", metadata={'page': 0, 'source': '문무대왕함.pdf'}),
   573.1139526367188)],
 4)

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
chroma_client.delete_collection('my_collection')

In [ ]:
collection = chroma_client.create_collection(name="my_collection")

In [ ]:
type(collection)

chromadb.api.models.Collection.Collection

In [ ]:
# https://stackoverflow.com/questions/76379440/how-to-see-the-embedding-of-the-documents-with-chroma-or-any-other-db-saved-in

collection.get(include=['embeddings'])

{'ids': ['aux-002'],
 'embeddings': [[-1.0233348608016968,
   0.2500264346599579,
   0.8553662300109863,
   0.9897406697273254,
   -0.2929781675338745,
   1.3080483675003052,
   -0.4714702069759369,
   0.32181987166404724,
   -0.48848283290863037,
   -0.020700603723526,
   -0.12246314436197281,
   0.47190389037132263,
   0.6336960196495056,
   -1.75662362575531,
   0.7461000084877014,
   0.3761701285839081,
   1.3289819955825806,
   0.282390296459198,
   -0.2723201811313629,
   0.09921205043792725,
   0.5150052309036255,
   -0.49155595898628235,
   0.7654712796211243,
   -0.37658819556236267,
   0.4456285238265991,
   -0.4344286024570465,
   0.6014817357063293,
   -0.4521670639514923,
   -1.2871569395065308,
   -0.6396574974060059,
   -0.8254866003990173,
   0.6039325594902039,
   1.0259079933166504,
   0.6400608420372009,
   0.16921621561050415,
   0.7477097511291504,
   0.06082267686724663,
   0.7571563720703125,
   -0.539139449596405,
   0.5155027508735657,
   0.6817148327827454,
  

### local saving and loading

In [ ]:
# local에 저장하면서 객체 생성
# 이미 폴더가 있으면 레코드를 계속 누적하게 됨
Chroma().delete_collection()
chroma_db2 = Chroma.from_documents(docs_for_chroma, embeddings,
                                   persist_directory="./chroma_db")

# 디스크로부터 로딩
new_db = Chroma(persist_directory="./chroma_db",
             embedding_function=embeddings)

In [ ]:
query = "문무대왕함의 특징은?"

# score = 1 - cos(theta) 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = new_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs, len(search_docs)

([(Document(page_content="오늘 '문무대왕함 '의 탄생으로  우리 해군은  활동영역이  더욱 넓어지게  되었습니다 . 遠海에서의  작\n전능력이  크게 향상되고 , 입체적인  대함, 대잠, 대공 작전능력도  한층 강화됩니다 . 최첨단  무기체\n계와 스텔스  기능, 그리고  최신의  자동화된  지휘통제  시스템도  '문무대왕함 '의 자랑입니다 .", metadata={'page': 0, 'source': '문무대왕함.pdf'}),
   279.5512041779731),
  (Document(page_content='친애하는  해군장병  여러분 , \n \n그리고  현대중공업  임직원과  내외 귀빈 여러분 , \n \n오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최\n신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의', metadata={'page': 0, 'source': '문무대왕함.pdf'}),
   297.860968129849),
  (Document(page_content="대한민국  해군의  무궁한  발전과  '문무대왕함 '의 武運長久를  기원합니다 .", metadata={'page': 1, 'source': '문무대왕함.pdf'}),
   303.2063426730197),
  (Document(page_content="동아시아의  바다를  제패했던  장보고  대사의  자랑스런  역사가  있습니다 . 이제는  우리가  그 역사를  \n새롭게  이어가야  합니다 . 5대양으로  항진해  나아가는  '문무대왕함 '에는 대양해군의  웅대한  비전이  \n담겨 있습니다 . 삼국통일의  위업을  이룩하고  죽어서도  나라를  지켜내겠다던  문무대왕의  기개와", metadata={'page': 1, 'source': '문무대왕함.pdf'}),
   343.94031276973635)],
 4)

## FAISS

- 고밀도 벡터의 효율적인 유사도 검색 및 클러스터링을 위한 라이브러리

- 일반적인 개념의 DB는 아니고 백터 저장과 검색에 특화

- https://python.langchain.com/docs/integrations/vectorstores/faiss

- https://faiss.ai/

### 텍스트 로딩

In [5]:
# pdf 파일을 페이별로 잘라서 로딩
loader= PyPDFLoader("문무대왕함.pdf")
pdf_pages = loader.load_and_split()

# 페이지별로 읽혀진 pdf를 다시 문서조각으로 자름
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=0)
docs_for_faiss = text_splitter.split_documents(pdf_pages)

In [15]:
type(docs_for_faiss)

list

### 벡터 db 생성 및 유사도 검색

In [6]:
from langchain_community.vectorstores import FAISS

In [13]:
faiss_db = FAISS.from_documents(docs_for_faiss, embeddings)

In [ ]:
query = "문무대왕함의 특징은?"

# score = L2 Distance 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = faiss_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs, len(search_docs)

([(Document(page_content="오늘 '문무대왕함 '의 탄생으로  우리 해군은  활동영역이  더욱 넓어지게  되었습니다 . 遠海에서의  작\n전능력이  크게 향상되고 , 입체적인  대함, 대잠, 대공 작전능력도  한층 강화됩니다 . 최첨단  무기체\n계와 스텔스  기능, 그리고  최신의  자동화된  지휘통제  시스템도  '문무대왕함 '의 자랑입니다 .", metadata={'source': '문무대왕함.pdf', 'page': 0}),
   279.5512),
  (Document(page_content='친애하는  해군장병  여러분 , \n \n그리고  현대중공업  임직원과  내외 귀빈 여러분 , \n \n오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최\n신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의', metadata={'source': '문무대왕함.pdf', 'page': 0}),
   297.86093),
  (Document(page_content="대한민국  해군의  무궁한  발전과  '문무대왕함 '의 武運長久를  기원합니다 .", metadata={'source': '문무대왕함.pdf', 'page': 1}),
   303.20633),
  (Document(page_content="동아시아의  바다를  제패했던  장보고  대사의  자랑스런  역사가  있습니다 . 이제는  우리가  그 역사를  \n새롭게  이어가야  합니다 . 5대양으로  항진해  나아가는  '문무대왕함 '에는 대양해군의  웅대한  비전이  \n담겨 있습니다 . 삼국통일의  위업을  이룩하고  죽어서도  나라를  지켜내겠다던  문무대왕의  기개와", metadata={'source': '문무대왕함.pdf', 'page': 1}),
   343.94034)],
 4)

### local saving and loading

In [ ]:
faiss_db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)

In [ ]:
query = "문무대왕함의 특징은?"

# score = L2 Distance 값 작을 수록 같은 곳을 지향하는 벡터
search_docs = new_db.similarity_search_with_score(query)  # with_score 없으면 그냥 문서만 반환
search_docs, len(search_docs)

([(Document(page_content="오늘 '문무대왕함 '의 탄생으로  우리 해군은  활동영역이  더욱 넓어지게  되었습니다 . 遠海에서의  작\n전능력이  크게 향상되고 , 입체적인  대함, 대잠, 대공 작전능력도  한층 강화됩니다 . 최첨단  무기체\n계와 스텔스  기능, 그리고  최신의  자동화된  지휘통제  시스템도  '문무대왕함 '의 자랑입니다 .", metadata={'source': '문무대왕함.pdf', 'page': 0}),
   279.5512),
  (Document(page_content='친애하는  해군장병  여러분 , \n \n그리고  현대중공업  임직원과  내외 귀빈 여러분 , \n \n오늘 우리는  대양해군의  미래를  열어나가는  데 또 하나의  이정표를  세웠습니다 . 해군의  최대, 최\n신예 구축함인  ‘문무대왕함 ’의 진수를  진심으로  축하합니다 . ‘문무대왕함 ’의 저 위용은  대한민국의', metadata={'source': '문무대왕함.pdf', 'page': 0}),
   297.86093),
  (Document(page_content="대한민국  해군의  무궁한  발전과  '문무대왕함 '의 武運長久를  기원합니다 .", metadata={'source': '문무대왕함.pdf', 'page': 1}),
   303.20633),
  (Document(page_content="동아시아의  바다를  제패했던  장보고  대사의  자랑스런  역사가  있습니다 . 이제는  우리가  그 역사를  \n새롭게  이어가야  합니다 . 5대양으로  항진해  나아가는  '문무대왕함 '에는 대양해군의  웅대한  비전이  \n담겨 있습니다 . 삼국통일의  위업을  이룩하고  죽어서도  나라를  지켜내겠다던  문무대왕의  기개와", metadata={'source': '문무대왕함.pdf', 'page': 1}),
   343.94034)],
 4)